In [1]:
import xarray as xr
import os
import pandas as pd

ds = xr.open_dataset('/home/sebastian/Documentos/analisisNETCDF/años/2021/IVT_2021_01.nc')
df = ds.to_dataframe()
# Filtrar valores mayores a 250
df_filtered = df.loc[df['IVT'] > 250]

# Resetear el índice y agregar el número como índice al principio
df_filtered.reset_index(inplace=True)
df_filtered.index += 1

# Renombrar el nombre de la columna de índice
df = df_filtered.rename_axis("Index", axis="columns")

# Imprimir el DataFrame
print(df)

Index                 time  latitude  longitude         IVT
1      2021-01-01 00:00:00      14.5     -85.50  289.593628
2      2021-01-01 00:00:00      14.5     -84.75  335.406342
3      2021-01-01 00:00:00      14.5     -84.00  330.467621
4      2021-01-01 00:00:00      14.5     -83.25  333.636383
5      2021-01-01 00:00:00      14.5     -82.50  330.948425
...                    ...       ...        ...         ...
341733 2021-01-31 18:00:00     -62.0     -63.75  371.662598
341734 2021-01-31 18:00:00     -62.0     -63.00  340.531952
341735 2021-01-31 18:00:00     -62.0     -62.25  307.716156
341736 2021-01-31 18:00:00     -62.0     -61.50  279.961853
341737 2021-01-31 18:00:00     -62.0     -60.75  251.652512

[341737 rows x 4 columns]


In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

def connected_component_label(path):
    # Obtener la imagen de entrada
    img = cv2.imread(path, 0)

    # Convertir los píxeles con valores de 1 a 127 a 0 y los demás a 1
    img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)[1]

    # Aplicar cv2.connectedComponents()
    num_labels, labels = cv2.connectedComponents(img)

    # Mapear las etiquetas de los componentes al valor de tono (hue)
    # El rango de tono en OpenCV es 0-179
    label_hue = np.uint8(179 * labels / np.max(labels))
    blank_ch = 255 * np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

    # Convertir a espacio de color BGR
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)

    # Establecer las etiquetas de fondo (background) en blanco
    labeled_img[label_hue == 0] = 255

    # Filtrar componentes pequeños (líneas delgadas)
    min_component_size = 1000  # Umbral mínimo de tamaño del componente
    large_component_mask = np.zeros_like(labels, dtype=np.uint8)
    large_component_mask[np.where(labels > 0)] = 255

    # Aplicar una operación de apertura para eliminar elementos pequeños y estrechos
    kernel = np.ones((5, 5), np.uint8)
    opened_img = cv2.morphologyEx(large_component_mask, cv2.MORPH_OPEN, kernel)

    # Encontrar los contornos de los componentes
    contours, _ = cv2.findContours(opened_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Crear una imagen de fondo blanco
    contour_img = 255 * np.ones_like(labeled_img)

    # Definir una lista de colores distintivos
    distinctive_colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255), (0, 255, 255)]

    # Filtrar los contornos más grandes
    large_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_component_size]

    # Dibujar solo los contornos grandes con colores distintivos
    for i, contour in enumerate(large_contours):
        color = distinctive_colors[i % len(distinctive_colors)]  # Elegir un color distintivo de la lista

        # Dibujar el contorno exterior
        cv2.drawContours(contour_img, [contour], -1, color, -1)

        # Dibujar el fondo blanco del objeto correspondiente
        cv2.drawContours(contour_img, [contour], -1, (255, 255, 255), thickness=cv2.FILLED)

    # Establecer el fondo negro como blanco
    contour_img[np.where((contour_img == [0, 0, 0]).all(axis=2))] = [255, 255, 255]

    # Guardar la imagen en la carpeta "objetos"
    output_dir = "objetos"
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, "contornos.png")
    cv2.imwrite(output_path, contour_img)

    # Devolver la imagen filtrada y el número de etiquetas
    return contour_img, num_labels

# Ruta de la imagen
path = "/home/sebastian/Documentos/analisisNETCDF/imagenBinFinal/imagen_final.png"

# Llamar a la función connected_component_label
filtered_img, num_labels = connected_component_label(path)
